In [ ]:
# standard python utilities
import os
import sys
import glob
import pandas as pd
import numpy as np
import calendar
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

# modflow utility
import flopy
import flopy.utils.binaryfile as bf


In [ ]:
## Set up directory referencing
# Package data
git_dir = os.path.dirname(os.path.dirname(os.getcwd()))
git_dir = os.getcwd()
while os.path.basename(git_dir) != 'CosumnesRiverRecharge':
    git_dir = os.path.dirname(git_dir)
gwfm_dir = '\\'.join(str.split(git_dir,'\\')[0:3])+ '/Box/research_cosumnes/GWFlowModel'
print(git_dir, gwfm_dir)

In [ ]:
# set box directory for output figures and data
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

# tprogs_id = '' # original tprogs with conditioning data in output tsim
tprogs_id = '_no_conditioning'
# tprogs_id = 'north_shifted_stream_distance_analysis'


data_dir = box_dir+ tprogs_id+'/data_output/'
fig_dir = box_dir+tprogs_id+'/figures/'


In [ ]:
percentile = 95
label='regional'
hf_tot_df = pd.read_csv(data_dir+'surface_highflow_by_distance_'+label+'_'+str(percentile)+'.csv')
label='local'
# each set of 100 rows is a different location
hf_tot_df_local = pd.read_csv(data_dir+'surface_highflow_by_distance_'+label+'_'+str(percentile)+'.csv')
# join local and regional datasets
hf_all = hf_tot_df.append(hf_tot_df_local)
# create clean index for easier referencing
site_order = ['Regional','Blodgett','Mahon','Oneto-Denier']
site_realization = np.vstack((np.repeat(site_order,100),
                              np.tile(np.arange(1,101),int(hf_all.shape[0]/100))))
site_realization_index = pd.MultiIndex.from_arrays(site_realization,names=['Site','Realization'])
hf_all = hf_all.set_index(site_realization_index)
# convert from number of cells to area, m^2
hf_all.loc[:] = hf_all.values*200*200
# plot with setback as x and realizations as label
hf_all.columns = pd.to_numeric(hf_all.columns)

## Cost Analysis
When looking at the local scale there is much greater variability in the setback distances where the maximum gradient appears. I can plot at which setback distance the gradient occurs and calculate the standard deviation for the three local sites and on the regional scale. I can also plot the variability in the area encompassed by the maximum gradient.  

Calculate XS area gain vs lost after setback to see if volume lost equals volume gained which means no fill dirt purchase is necessary. Then assume excavation cost for only half of it (elevation before > elevation after). This does not account for the need to build levees at a setback distance...  

Average cost to excavate land is 50-200 \\$/cubic yard. Imagine a XS of levee is 300 yd^2, 6000\\$/yd of levee or 2000\\$/ft which gives 10,000,000\\$/mile of levee just to move the earth. This aligns with estimates of levee setback cost of 2.5-5M\\$ in washington state to up to 20M\\$ in Sacramento. According to acrevalue.com the average cost of an acre of farmland in Sac county is 20K\\$/acre. Per mile of 3200 m levee setback in both directions requires 2,500 acres of land which would put an average land cost of \\$51,000,000, at 400m back we would see 6,400,000

I need to calculate what is considered diminishing returns. At what rate of area gained by distance setback is it not worth it? I could do some calculations on the cost benefit of recharge and the cost of the land and setback
Each AF recharged equals about 4 ft of head increase in that local area, should you consider the value of recharge as reduction in pumping cost or actual cost of water saved for future; I lean toward water saved cost because we need to replenish aquifers to prevent overdraft, but I could see farmers/ag-res only viewing it as reduction in pumping energy because they assume groundwater will naturally maintain itself.  
The cost analysis should be done on the 1-2 year flood scale (2000 cfs) and the 20 year flood scale (10,000 cfs) to show water recharged typically and benefit from less frequent big events. Because if 600k $ is done every year that is substantial, but more likely that would only occur during 20 year events with big recharge volumes.  
Optimization looks along the setback distance vs high recharge area plot following these rules: 
1. Setback distance must be >= 0 and <= 3200 m
2. Maximize the function benefit = dollars of water recharged - cost of setback and land  

If I want to publish in an engineering journal then I need to include cost, if I'm seeking for more general scientific advancement then I should not directly include cost. The issue I'm having without cost is finding a way to pick the best setback distance with derivatives or maximum returns.  

gravel K = 4E-3 m/s
sand K = 1.5E-3 m/s  
**GW Pumping energy cost**  
C = 0.746 Q h c / (3960 μp μm)  
C = cost per hour (USD/hour, EUR/hour, ...)  
Q = volume flow (US gpm), h = differential head (ft)  
c = cost rate per kWh (USD/kWh, EUR/kWh, ....)  
μp = pump efficiency (0 - 1), μm= motor efficiency (0 - 1)  

In [ ]:
# Q = 500 # gpm, average between domestic 100 and ag 1000
h = 80 # ft, typical DTW in basin
c = 0.24 # USD/kWh, pge lists 24cents as low, 34 as tier 2
ep = 0.9
em = 0.9
C_gpm_1 = 0.746 *h* c*24 / (3960 *ep *em) # USD/day/gpm
C_AF_1 = C_gpm_1/(1/7.48)/(1/43560)/(60*24) #USD/day/AFd
h= 76
C_gpm_2 = 0.746 *h* c*24 / (3960 *ep *em) # USD/day/gpm
C_AF_2 = C_gpm_2/(1/7.48)/(1/43560)/(60*24) #USD/day/AFd
print('Cost per acre foot to pump GW for SASb: $%.2f '%C_AF_1) # USD/day/AFd or USD/AF
print('Cost per acre foot change per 1 ft recharge for SASb: $%.2f '%(C_AF_1-C_AF_2)) # USD/day/AFd or USD/AF
C_AF = C_AF_1

To really flesh this analysis out it would be interesting to see what ecosystem benefit there is (some dollar ammount equivalent might exist). This would require particle tracking from each recharge cell under steady state to determine the percentage that travels to groundwater and the percentage that return to the river. I could estimate this by looking at how much TNC pays to keep water rights per AF to keep flow in a river and estimating how much of the recharge would return as baseflow.

In [ ]:
from scipy.stats import gmean
# K m/s
K_grv = 4E-3
K_snd = 1.5E-3
# take geometric mean and convert to m/day
K_coarse = gmean([K_grv,K_snd])*86400
z_aniso = 100 # log between 100, 1000
K_coarse /= z_aniso
K_coarse



In [ ]:
# hf_tot_df_local.iloc[0:100].mean(axis=0).plot()
# mean number of cells for all setbacks for each realization
mean_area = hf_tot_df_local.iloc[0:100].mean(axis=1)*200*200
# m3/day
m3_AF = (1/0.3048**3)*(1/43560)
# looks like 200 AF per day is doable for most realizations, also would expect the flooding to happen for may 1-2 weeks per year
(K_coarse*mean_area*m3_AF*C_AF/1000).plot()
plt.ylabel('Estimated Recharge Benefit\n (thousand dollars per day)')
plt.xlabel('Realization')
plt.show()

# cost of surface water in sacramento valley $50-100/AF
# cost of pumping?

(K_coarse*mean_area*m3_AF*C_AF*14/1000).plot()
plt.ylabel('Estimated Recharge Benefit\n (thousand dollars per year)')
plt.xlabel('Realization')

Start with the best case scenario that every year every high recharge cell in the setback is activated with 0.5 meters of water (or calculated flood depth) which is typical of the flatter cross-sections. Then assume a cost of $50/AF water recharged and annualize the cost. Annualize cost of levee construction then perform cost benefit optimization.

In [ ]:
local_setback = gpd.read_file(gwfm_dir+'/levee_setback/local_levee_setback_rectangles/local_levee_setback_rectangles.shp')
local_setback = local_setback.dropna()

In [ ]:
local_setback.geometry.iloc[0].area
for n in np.arange(0,len(local_setback)):
    xy = list(local_setback.geometry.iloc[n].exterior.coords)
    xy_dists = np.sum(np.sqrt(np.diff(xy,axis=0)**2),axis=1)
    local_setback.loc[n,'width_m'] = np.min(xy_dists)


In [ ]:
const_cost_m3 = 200 # 200$/m3 or yd3
land_cost_m2 = 20000/43560/0.3048 # convert from cost per acre to cost per sq. ft then to sq m



In [ ]:
const_int = 0.08 # higher interest rate for productive investments and lets assume levees are built for 100 years
num_pers = 100 # number of periods, assume interest is compounding annually
x_dist_m = 400 # setback distance total, increments of 400 from 400 to 6400 m
const_cost = 2000/0.3048 # approx cost to move earth per meter of levee times total length (62 km) or local length (~2250 m)
# land_cost = 2000/0.3048 # approx cost to move earth per meter of levee times total length (62 km) or local length (~2250 m)
total_len = 62000
local_len = 2250

annual_cost = (const_cost*total_len + land_cost_m2*x_dist_m*total_len)*(const_int/(1-(1+const_int)**(-num_pers)))

const_int, 0.08/12
print('Annualized cost for all work is %.2f M$' %(annual_cost/1E6))
# annual_cost = const_cost*2250

In [ ]:
const_int = 0.08 # higher interest rate for productive investments and lets assume levees are built for 100 years
num_pers = 100 # number of periods, assume interest is compounding annually
x_dist_m = 400 # setback distance total, increments of 400 from 400 to 6400 m
const_cost = 2000/0.3048 # approx cost to move earth per meter of levee times total length (62 km) or local length (~2250 m)
# land_cost = 2000/0.3048 # approx cost to move earth per meter of levee times total length (62 km) or local length (~2250 m)

annual_cost = (const_cost*local_len + land_cost_m2*x_dist_m*local_len)*(const_int/(1-(1+const_int)**(-num_pers)))

const_int, 0.08/12
print('Annualized cost for all work is %.2f M$' %(annual_cost/1E6))
# annual_cost = const_cost*2250

because the cost so much greatly outweighs any potential benefit, rather than maximing profit we will focus on minimizing cost

In [ ]:
def annual_cost(x, df):
    x = int(x)
    x_dist_m = np.arange(0,6800,400)[x]
    annual_cost = (const_cost*local_len + land_cost_m2*x_dist_m*local_len)*(const_int/(1-(1+const_int)**(-num_pers)))
    # N rch_rate *area * rch_depth * AF conv * cost/AF
    annual_benefit = (df.iloc[x]*200*200*0.5)*C_AF*m3_AF
    net_cost = annual_cost - annual_benefit
    return(net_cost)

In [ ]:
# C_AF = 1500
C_AF=50

In [ ]:
from scipy.optimize import fsolve, least_squares, minimize, minimize_scalar
# fsolve(mannings, [2], args = (Q_cms, xs_elevs), xtol=1E-5)
# least_squares(mannings, [2], args = (Q_cms, xs_elevs), bounds = (0, 10), ftol=1E-5)
# minimize(mannings, [1.], args = (Q_cms, xs_elevs), bounds=[(0,10)], tol=1E-5)

res = minimize_scalar(annual_cost,  args = (hf_tot_df.iloc[0]), bounds=(0,17), method='bounded')
res


In [ ]:
rch_depth = 2 # assume all water flooding infiltrates and becomes groundwater with value to be pumped

In [ ]:

x_dist_m = np.arange(0,6800,400)
annual_cost = (const_cost*total_len + land_cost_m2*x_dist_m*total_len)*(const_int/(1-(1+const_int)**(-num_pers)))
annual_benefit = (hf_all.loc['Regional']*rch_depth)*C_AF*m3_AF 
net_cost = annual_cost - annual_benefit
net_cost.transpose().plot(legend=False, cmap='gray')

# print(annual_cost.mean(), annual_benefit.mean().median())
print('Annual cost mean %.2E ' %annual_cost.mean(), ' annual benefit mean %.2E' %annual_benefit.mean().median())

plt.ylabel('Net Annual Cost ($)')
plt.title('Total Length (62 km)')
# on a regional scale this will never be cost effective, even at 1000$/AF the zero setback is mostly the minimum
# at $1500/AF there are minimums around 1200 m setback

In [ ]:

x_dist_m = np.arange(0,6800,400)
annual_cost = (const_cost*local_len + land_cost_m2*x_dist_m*local_len)*(const_int/(1-(1+const_int)**(-num_pers)))
annual_benefit = (hf_all.loc['Mahon']*rch_depth)*C_AF*m3_AF 
net_cost = annual_cost - annual_benefit
net_cost.transpose().plot(legend=False,cmap='gray')

plt.xlabel('Setback distance (m)')
plt.ylabel('Net cost ($)')
plt.title('Avg Local Length (2250 m)')

print('Annual cost mean %.2E ' %annual_cost.mean(), ' annual benefit mean %.2E' %annual_benefit.mean().median())

# as the recharge depth increases we see more variability in the plots and the minimum
# values are no longer at a zero setback distances
# I could plot minimum values at different recharge rates to avoid the flood mapping, but I think
# flood mapping is the best way

In [ ]:
annual_benefit/annual_cost

In [ ]:
50*m3_AF # the value of water ends of 0.05 dollars per m^3 which is similar to Thomas' values with Mehrdad
 # area of recharge times depth (assumed 0.5 m now) * values/volume and assume yearly
yearly_value = (hf_tot_df*200*200*0.5)*50*m3_AF
# iterate through each realization
yearly_value.iloc[0,:]